**Importamos las librerías a utilizar**

In [206]:
#Importamos libreria a usas
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('Librerias importadas.')

Librerias importadas.


**Importando datos de Toronto de Wikipedia**


In [137]:
ruta = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
listar = pd.read_html(ruta)
df_toronto = listar[0]
import numpy as np
df_toronto['Borough'].replace('Not assigned',np.nan,inplace = True)
df_toronto.dropna(subset=['Borough'], axis = 0, inplace = True)
df_toronto.reset_index(inplace=True)
df_toronto.drop(columns = {'index'},inplace = True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Aqui vamos a resolver el punto:**

Puede existir más de un vecindario en un área de código postal. 
Por ejemplo, en la tabla de la página de Wikipedia, notará que M5A aparece dos veces y tiene dos vecindarios: 
Harbourfront y Regent Park . Estas dos filas se combinarán en una fila con los barrios separados con una coma como se muestra en la fila 11 de la 
tabla anterior.

In [138]:
#Validamos si existen duplicados
df2 = df_toronto['Postal Code'].duplicated()
df2.value_counts()#Hay 103 resultados como False lo que indica que no hay duplicados

False    103
Name: Postal Code, dtype: int64

In [139]:
#Nos cercioramos de todos modos con el Cod Postal que ponen como ejemplo M5A
df2 = df_toronto[df_toronto['Postal Code']=='M5A']
df2 # Como se observa solo nos devuelve un registro

,Postal Code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Si una celda tiene un distrito pero no un vecindario asignado , el vecindario será el mismo que el distrito.**

In [140]:
df_T = df_toronto[df_toronto['Neighborhood']=='Not assigned']
df_T #Vemos que no ay vencindarios como Not assigned

,Postal Code,Borough,Neighborhood


In [142]:
df_toronto.shape

(103, 3)

# **Cargamos el archivo CSV que contiene las coordenadas geográficas**

In [168]:
coor = '/home/Geospatial_Coordinates.csv'
df_coor = pd.read_csv(coor)
df_coor

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


**Juntamos** los 2 DF

In [171]:
df_toronto_merge = df_toronto
df_toronto_merge = df_toronto_merge.join(df_coor.set_index('Postal Code'), on='Postal Code')
df_toronto_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Usamos geocode para saber las coordenadas de **Toronto**

In [172]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordenadas Geograficas {}, {}.'.format(latitude, longitude))

Coordenadas Geograficas 43.6534817, -79.3839347.


Unicamos en el mapa a Toronto y marcamos los **vencidarios**

In [196]:
mapa_toronto = folium.Map(location=[latitude,longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_merge['Latitude'], df_toronto_merge['Longitude'], df_toronto_merge['Borough'], df_toronto_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa_toronto)  
mapa_toronto

**Utilizaremos la API de Foursquare para explorar los vecindarios y segmentarlos.**

In [0]:
#Definimos credenciales
CLIENT_ID = 'MPS5EM5OHF5W40AKIFBJV2W4RG3C2L2KJOMAEKBMZ3LQLJHZ'
CLIENT_SECRET = 'IO3SXEE3XX5TWAOFKVZJP2Z5GNFNF1O2WCTXAAFBSCAE2CS4'
VERSION = '20180605'

**Creamos una funcion para obtener los 100 mejores lugares que se encuentran a 500 metros de cada barrio**

In [0]:
LIMIT = 500
def obtener_lugares(names, latitudes, longitudes, radius=500):
    
    lista_lugar=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Creamos el API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # Realizamos la solicitud
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Retornamos información relevante
        lista_lugar.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    df_lugares = pd.DataFrame([item for lista_lugar in lista_lugar for item in lista_lugar])
    df_lugares.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(df_lugares)

**Pasamos los parametros a la función para que nos devuelva el df con los lugares mas cercanos**

In [0]:
df_toronto_lugares = obtener_lugares(names=df_toronto_merge['Neighborhood'],
                                   latitudes=df_toronto_merge['Latitude'],
                                   longitudes=df_toronto_merge['Longitude'])

**Validamos cuantos datos tenemos y mostramos los 5 primeros registros**

In [208]:
print(df_toronto_lugares.shape)
df_toronto_lugares.head()

(2132, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


**Verificamos cuantos lugares nos devuelve por cada vecindario**

In [209]:
df_toronto_lugares.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
